In [1]:
from tqdm import tqdm

from User import User
from Survey import Survey
from Intake import Intake

import pandas as pd

In [2]:
def rename_columns(old_name:str, new_name: str, df: pd.DataFrame) -> None:
    df.rename(columns = { old_name : new_name }, inplace=True)
    return None

# Breakdown (From user to ingredient):

The file has many users.

Each user has many surveys.

Each survey has many meal intake.

Each intake consists of many food components.

Every food component is marked with a "Nutrition ID code".

In [3]:
# Rename the columns (For Intake24)

intake24_df = pd.read_csv('files/intake24_survey_file.csv')

#print(intake24_df.columns)

rename_columns('Energy, with dietary fibre', 'energy_with_fibre', intake24_df)
rename_columns('Meal name', 'meal_name', intake24_df)
rename_columns('Survey ID', 'survey_id', intake24_df)
rename_columns('Intake24 food code', 'food_code', intake24_df)
rename_columns('User ID', 'user_id', intake24_df)
rename_columns('Meal ID', 'meal_id', intake24_df)
rename_columns('Nutrient table code', 'heifa_nutrient_id', intake24_df)
rename_columns('Portion size (g/ml)', 'portion_size_consumed', intake24_df)

#print("\n\n")
#print(intake24_df.columns)

In [4]:
# Structure is like this:
# - Map user ID to their respective meal and information
# - Map the meal number to the respective ingredients
# - Map the ingredients to their respective nutrient code

user_dict = {}

In [5]:
def loop_ingredients(food_df: pd.DataFrame) -> dict:

    meal_intake = Intake()
    meal_intake.add_food_information(food_df)
    
    return meal_intake

def loop_meals(meal_ids_list: list, survey_meals_df: pd.DataFrame) -> None:

    survey_info_meals = Survey()
    
    for meal_id in meal_ids_list:
        
        food_intake_df = survey_meals_df.query("meal_id == @meal_id")
        meal_ingredients = loop_ingredients(food_intake_df)

        # To be removed and replaced with OOP
        survey_info_meals.add_meal(meal_id, meal_ingredients)

    return survey_info_meals

In [6]:
# Step 1: Let's split by the survey ID.

# Get the list of survey IDs.
survey_ids_list = intake24_df['survey_id'].unique().tolist()

# Loop one survey ID at a time and split the Intake24 dataframe accordingly.
for survey_id in tqdm(survey_ids_list, ncols=50):

    survey_meals_df = intake24_df.query("survey_id == @survey_id")

    # Extract the user ID.
    # If it exists, extract the object.
    # Otherwise, create a new object
    user_id = survey_meals_df['user_id'].values[0]
    
    if user_id not in user_dict:
        user_dict[user_id] = User(user_id)

    user = user_dict[user_id]
    
    # Step 2: Split further the Meal ID
    meal_ids_list = survey_meals_df['meal_id'].unique().tolist()

    # Step 3: Get information of the meals
    survey_meals_info = loop_meals(meal_ids_list, survey_meals_df)

    # Every meal of the survey will be populated here
    user.add_survey(survey_id, survey_meals_info)

100%|███████████| 405/405 [00:28<00:00, 14.21it/s]


In [7]:
for user_id in user_dict.keys():

    #print(f"Printing for User {user_id}")
    user_obj = user_dict[user_id]
    user_obj.print_information()

Printing for Survey ID 5d3e72da-a241-4e8e-837b-021accf9a37c

HEIFA code(s) for Breakfast: ['04B10082', '02B10604', '09A10204', '01B10301']

Printing for 04B10082:
 Portion size of 15.5g/ml
 Energy (Dietary Fibre included): 357.9

Printing for 02B10604:
 Portion size of 54.0g/ml
 Energy (Dietary Fibre included): 638.28

Printing for 09A10204:
 Portion size of 16.42g/ml
 Energy (Dietary Fibre included): 32.5

Printing for 01B10301:
 Portion size of 194.92g/ml
 Energy (Dietary Fibre included): 11.7

HEIFA code(s) for Lunch: ['08E30320', '09A10204', '01B10301']

Printing for 08E30320:
 Portion size of 51.75g/ml
 Energy (Dietary Fibre included): 918.05

Printing for 09A10204:
 Portion size of 16.42g/ml
 Energy (Dietary Fibre included): 32.5

Printing for 01B10301:
 Portion size of 195.2g/ml
 Energy (Dietary Fibre included): 11.71

HEIFA code(s) for Afternoon snack or drink: ['04B10082', '02B10604', '09A10204', '01B10301']

Printing for 04B10082:
 Portion size of 12.4g/ml
 Energy (Dietary Fi

# HEIFA - Ingredients

Every row in the file is a unique ingredient.

Every ingredient:
- has it's own attributes.
- can be mapped to a 8-digit code (for HEIFA Recipe)
- is used as a divisor for either energy (kilo joules) or grams (g)

In [ ]:
heifa_ingred_df = pd.read_csv('files/'